## Preprocessing

In [156]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [157]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"], axis=1, inplace=True)

In [158]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [159]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_type = application_df["APPLICATION_TYPE"].value_counts()
app_type 

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [160]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# all application types with a count lower than 1000 will be treated is outliers and will be rolled up into other
application_types_to_replace = list(app_type[app_type < 1000].keys())

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [161]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [162]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification = application_df["CLASSIFICATION"].value_counts()
classification[classification > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [163]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# all classifications with a count lower than 2000 will be treated is outliers and will be rolled up into other
classifications_to_replace = list(classification[classification < 2000].keys())

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: count, dtype: int64

In [164]:
# Convert categorical data to numeric with `pd.get_dummies`
cate_columns = list(application_df.dtypes[application_df.dtypes == object].keys())
application_dummies = pd.get_dummies(application_df, columns=cate_columns, dtype=int)

In [165]:
# Split our preprocessed data into our features and target arrays
# Will also be dropping the status column as a company being active now should not have high impact on its loan approvals in the past.
y = application_dummies["IS_SUCCESSFUL"]
X = application_dummies.drop(columns=["IS_SUCCESSFUL", "STATUS"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

In [166]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [167]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=len(X.columns), activation="relu", input_dim=len(X.columns)))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=22, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=11, activation="relu"))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

c:\Users\cadkm\anaconda3\envs\inclassfeb2024\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_42 (Dense)                │ (None, 37)             │         1,406 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 22)             │           836 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 11)             │           253 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 4)              │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,548 (9.95 KB)

 Trainable params: 2,548 (9.95 KB)

 Non-trainable params: 0 (0.00 B)

In [168]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [169]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 657us/step - accuracy: 0.6877 - loss: 0.6122
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - accuracy: 0.7263 - loss: 0.5629
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 628us/step - accuracy: 0.7267 - loss: 0.5606
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 649us/step - accuracy: 0.7356 - loss: 0.5506
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 633us/step - accuracy: 0.7338 - loss: 0.5550
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 644us/step - accuracy: 0.7309 - loss: 0.5532
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 666us/step - accuracy: 0.7324 - loss: 0.5533
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 648us/step - accuracy: 0.7357 - loss: 0.5513
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 655us/step - accuracy: 0.7338 - loss: 0.5510
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - accuracy: 0.7318 - loss: 0.5500
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 643us/step - accuracy: 0.7382 - loss: 0.5451
Epoch 12/100
804/80

KeyboardInterrupt: 

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 880us/step - accuracy: 0.7249 - loss: 0.5608
Loss: 0.560802698135376, Accuracy: 0.7248979806900024


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")